In [ ]:
import csv
import yaml
import pandas as pd
import openpyxl

from collections import OrderedDict
from scrapinghub import ScrapinghubClient

In [ ]:
# загружаем конфиг
with open('config.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [ ]:
def init_scrapinghub():
    api_key = config['scrapinghub']['api_key']
    project_id = config['scrapinghub']['project_id']

    client = ScrapinghubClient(api_key)
    project = client.get_project(project_id)

    return {'client': client, 'project': project}


def load_last_categories() -> list:
    """Export last two scraped WB categories for comparison"""
    sh = init_scrapinghub()

    jobs_summary = sh['project'].jobs.iter(has_tag=['daily_categories'], state='finished', count=2)

    counter = 0
    job_results = [[], []]

    for job in jobs_summary:
        job_results[counter] = []

        for item in sh['client'].get_job(job['key']).items.iter():
            job_results[counter].append({
                'wb_category_name': item['wb_category_name'],
                'wb_category_url': item['wb_category_url']
            })

        counter += 1

    return job_results


categories = load_last_categories()

# for details see https://pythondata.com/quick-tip-comparing-two-pandas-dataframes-and-getting-the-differences/
df1 = pd.DataFrame(categories[0], columns=['wb_category_name','wb_category_url'])
df2 = pd.DataFrame(categories[1], columns=['wb_category_name','wb_category_url'])

df = pd.concat([df1, df2]) # concat dataframes
df = df.reset_index(drop=True) # reset the index
df_gpby = df.groupby(list(df.columns)) #group by
idx = [x[0] for x in df_gpby.groups.values() if len(x) == 1] #reindex

print(len(idx))

ri = df.reindex(idx)

grouped = ri.groupby('wb_category_name', as_index=False).first()

grouped.to_excel('data/new_categories.xlsx', index=None, header=True)